## Scenario: Token Issuers can maintain multiple Token Status Lists

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import random

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /health)

In [3]:
health_endpoint = f"{BASE_URL}/health"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")
tc.assertEqual("OK", response.json().get("status"))

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/health
Health check successful! Server is running.


### Two Issuers register on the Status List Server by publish credentials

In [4]:
gondwana_digital_pole = fixtures.get_gondwana_digital_pole_issuer()
scott_holdings = fixtures.get_scott_holdings_issuer()

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    print("Proceeding with Issuer:", issuer_data.get('label'))
    
    credentials_endpoint = f"{BASE_URL}/credentials"
    print(f"Testing POST: {credentials_endpoint}")
    
    payload = {
      "issuer": issuer_data.get('label'),
      "public_key": issuer_data.get('keypair').get('public_key'),
      "alg": issuer_data.get('keypair').get('alg')
    }
    
    response = requests.post(credentials_endpoint, json=payload)
    tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
    print("Credentials published successfully!\n")

Proceeding with Issuer: gondwana-digital-pole-opydg
Testing POST: http://localhost:8000/credentials
Credentials published successfully!

Proceeding with Issuer: scott-holdings-k0e5s
Testing POST: http://localhost:8000/credentials
Credentials published successfully!



### They publish multiple status lists

In [5]:
# Since status lists are so far not linked to issuers, just assume they are.
# Below, each issuer publishes three lists.

status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    bearer_token = fixtures.create_bearer_jwt_token(issuer_data)
    for _ in range(3): # each issuer publishes three lists
        status_list_id = str(uuid.uuid4())
        print(f"{issuer_data.get('label')} publishes status list {status_list_id}")
        
        response = requests.post(status_publish_endpoint, json={
            "list_id": status_list_id,
            "status": [{"index": i, "status": "VALID"} for i in range(random.randint(1, 5))]
        }, headers={"Authorization": f"Bearer {bearer_token}"})
        
        tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")

Testing POST: http://localhost:8000/statuslists/publish
gondwana-digital-pole-opydg publishes status list 3cad99d6-b577-4e2f-8a93-b7c3266992cf
gondwana-digital-pole-opydg publishes status list 6b27ddee-6433-480a-9c63-9506d741594d
gondwana-digital-pole-opydg publishes status list bffa133d-55b7-4f88-bb68-232be923b87f
scott-holdings-k0e5s publishes status list 5288c1de-620c-4e79-a16b-ebb04d846aa4
scott-holdings-k0e5s publishes status list bbc9d578-ff98-4d3b-9886-df8e57a6d05a
scott-holdings-k0e5s publishes status list 8acdf452-7431-4db9-950b-4ef18c6148d9
